# Lab4: Visualizing trends from Order History

### Google Data Studio

Google Data Studio is a cloud powered service that can be used to deliver insights into your data by combining many different data sources to quickly visualize your data. Visualizations that are created can be made into dashboards that the team can collaborate on. 

### Retreiving the Cloud SQL MySQL Database Host 

In this lab we will be importing sample inventory data, and order history from a CSV into a MySQL Database in Google Cloud SQL. To begin, make sure you are signed into your Google account and can access the [Google Cloud Console](https://console.cloud.google.com/).

From the search bar at the top of the page, type in `SQL`. Click the **SQL** result to open the Cloud SQL console view.

Observe the **inventory-db** MySQL instance that has been created for you by the Cloud Academy lab environment:

![observe MySQL instance](assets/lab4/observe_instance.png)

It can take 10 minutes from when you started the lab for the instance to reach the ready status.

Copy the **Public IP address** and use it to overwrite the default `host` IP address in the code block in the following section.

#### Populating the Mock Data
Use the Python code block below to populate our mock data into the Cloud SQL Database.

In [7]:
! pip install mysql-connector

import mysql.connector, csv
## Make sure the Host matches your Cloud SQL instance
host="34.69.195.240"
mydb = mysql.connector.connect(
  host=host, 
  user="root",
  passwd="demotest123"
)

cursor = mydb.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS inventory_datastudio")
mydb.database = "inventory_datastudio"
cursor.execute("CREATE TABLE IF NOT EXISTS product_table (ProductName varchar(50),Description varchar(255),ProductSku bigint(12),QuantityInStock int(5),ReorderLevel int(5),WholeSaleCost decimal(8,2),RetailCost decimal(8,2));")
cursor.execute("CREATE TABLE IF NOT EXISTS order_history (CustomerName varchar(50),OrderDate varchar(255),OrderId bigint(12));")
cursor.execute("CREATE TABLE IF NOT EXISTS orders (OrderId bigint(12),ProductSku bigint(12), Price decimal(8,2));")
cursor.execute("CREATE VIEW orders_and_order_history AS SELECT t1.CustomerName, t1.OrderDate, t2.OrderId, t2.ProductSku, t2.Price FROM order_history t1, orders t2 WHERE t1.OrderId = t2.OrderId")

print("Database setup complete.")
with open('./data_files/Lab4_Inventory.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        cursor.execute ("INSERT INTO product_table (ProductName,Description,ProductSku,QuantityInStock,ReorderLevel,WholeSaleCost,RetailCost) VALUES (%s, %s, %s, %s, %s, %s, %s)",row)
    mydb.commit()
print("Success! Inventory Data inserted")

with open('./data_files/OrderHistory.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        cursor.execute ("INSERT INTO order_history (CustomerName, OrderDate, OrderId) VALUES (%s, %s, %s)",row)
    mydb.commit()
print("Success! Order History Data inserted")

with open('./data_files/Orders.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        cursor.execute ("INSERT INTO orders (OrderId, ProductSku, Price) VALUES (%s, %s, %s)",row)
    mydb.commit()
print("Success! Orders Data inserted")


mydb.close() 

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Database setup complete.
Success! Inventory Data inserted
Success! Order History Data inserted
Success! Orders Data inserted


#### **Lab:** Getting Started in GCP Data Studio
1. First [Navigate to Data Studio](https://datastudio.google.com/u/1/navigation/reporting)

2. Next click `Blank Report`  
![BlankReport](./assets/lab4/lab4_blank_report.png)

3. Navigate and Find `MySql` then enter the appropriate hostname, username, and password.  Select the orders_and_order_history view.  
![Authenticate](./assets/lab4/lab4_sql_authenticate.png)

4. Click `Add to Report` to add the data into the report.

5. Next we will create a quick report on which customer's had the most expensive orders. On right most bar you can select both the dimension and metric.  For the dimension select `Customer Name` and for the metric select `Sum - Price`  
![Report](./assets/lab4/lab4_dimension_metric.png)

6. Take a look at the results, we can see John Smith had the largest Order.  
![Order Chart](./assets/lab4/lab4_results.png)

Play around with some of the other dimensions to understand some insights of the order table.

#### **Lab:** Finding the Most Commonly Sold Product

In this lab we will add a separate table, perform data blending to join from the inventory table to the `orders_and_order_history` view, and find out which product we sell the most.

1) Select `Add Data` to add another table to Data Studio.

![Add Data](./assets/lab4/lab4_add_data.png)

2) Add a `MySql` data source.  Enter the database and connection details similar to above.

3) This time select `product_table` when it comes time to adding a table into the report.

4) Select `Blend Data` to open the blend data wizard.

![Blend Data](./assets/lab4/lab4_blend.png)

5) Click `Add Another Data Source` and add the recently added table from the previous steps.  Ensure the join keys have `ProductSku` and that the inventory table has `ProductName` included.  Ensure in the `orders_and_order_history` datastore has count selected for `ProductSku`

![datasource blending](./assets/lab4/lab4_datasources.png)

6) Modify the chart to have a dimension of `ProductName`, a metric of count for `ProductSku` , and a sort by `ProductSku` _descending_.

![graph creation](./assets/lab4/lab4_most_sold_products.png)

We have successfully created a chart that shows are most sold products!  Your graph should look similar to below.

![most sold products](./assets/lab4/lab4_productcount.png)

#### **Lab:** Other Insights on Data

1) Select `Add Data` to add another table to Data Studio.

![Add Data](./assets/lab4/lab4_add_data.png)

2) Select `My data sources` and find the previously added inventory data.

3) Modify the chart to find the most expensive product that is sold.

![Most expensive product](./assets/lab4/lab4_most_expensive.png)

4) Next we will find the total retail value of our inventory.  Select `Add a chart`

![add chart](./assets/lab4/lab4_add_chart.png)

5) Select `Scorecard`

6) Customize it to get the Sum of Retail Cost.

![sum chart](./assets/lab4/lab4_chart_sum.png)